In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, brier_score_loss, f1_score, fbeta_score, confusion_matrix
from tabpfn import TabPFNClassifier
from imblearn.over_sampling import SMOTENC
from sklearn.linear_model import LogisticRegression
from scipy.stats import norm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

def load_data(filepath):
    return pd.read_csv(filepath, sep=',', header=0)

# DeLong test
def delong_test(y_true, proba1, proba2):
    """
    Perform the DeLong test to compare the AUCs of two models.

    Args:
        y_true (array-like): True binary labels (0 or 1).
        proba1 (array-like): Predicted probabilities from model 1.
        proba2 (array-like): Predicted probabilities from model 2.

    Returns:
        z_score (float): Z-score for the difference in AUCs.
        p_value (float): Two-tailed p-value for the difference in AUCs.
    """

    def compute_midrank(x):
        """
        Compute the midrank of elements in the array x.

        Args:
            x (array-like): Input array.

        Returns:
            T2 (np.ndarray): Midrank values for the input array.
        """
        J = np.argsort(x)  # Indices that would sort the array
        Z = x[J]  # Sorted array
        N = len(x)  # Length of the array
        T = np.zeros(N, dtype=np.float64)  # Initialize midrank array

        # Compute midranks for tied values
        i = 0
        while i < N:
            j = i
            while j < N and Z[j] == Z[i]:  # Find the range of tied values
                j += 1
            T[i:j] = 0.5 * (i + j - 1)  # Assign midrank to tied values
            i = j

        # Reorder midranks to match the original array
        T2 = np.empty(N, dtype=np.float64)
        T2[J] = T + 1  # Add 1 to match R's rank convention
        return T2

    def compute_ground_truth_statistics(true):
        """
        Compute statistics needed for the DeLong test.

        Args:
            true (np.ndarray): True binary labels (0 or 1).

        Returns:
            order (np.ndarray): Indices that sort the true labels in descending order.
            label_1_count (int): Number of positive examples (label 1).
        """
        assert np.array_equal(np.unique(true), [0, 1]), "Ground truth must be binary."
        order = (-true).argsort()  # Sort indices in descending order of true labels
        label_1_count = int(true.sum())  # Count of positive examples
        return order, label_1_count

    # Ensure y_true is a numpy array
    true = np.array(y_true)

    # Compute ground truth statistics
    order, label_1_count = compute_ground_truth_statistics(true)

    # Stack probabilities and sort according to ground truth order
    sorted_probs = np.vstack((np.array(proba1), np.array(proba2)))[:, order]

    # Initialize arrays for midranks
    m = label_1_count  # Number of positive examples
    n = sorted_probs.shape[1] - m  # Number of negative examples
    k = sorted_probs.shape[0]  # Number of models (2 in this case)
    tx, ty, tz = [np.empty([k, size], dtype=np.float64) for size in [m, n, m + n]]

    # Compute midranks for positive, negative, and all examples
    for r in range(k):
        positive_examples = sorted_probs[r, :m]  # Probabilities for positive examples
        negative_examples = sorted_probs[r, m:]  # Probabilities for negative examples
        tx[r, :], ty[r, :], tz[r, :] = [
            compute_midrank(examples) for examples in [positive_examples, negative_examples, sorted_probs[r, :]]
        ]

    # Compute AUCs for each model
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n

    # Compute covariance components
    v01 = (tz[:, :m] - tx[:, :]) / n  # Covariance between positive and negative examples
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m  # Covariance between negative and positive examples

    # Compute the DeLong covariance matrix
    sx = np.cov(v01)  # Covariance for positive examples
    sy = np.cov(v10)  # Covariance for negative examples
    delongcov = sx / m + sy / n  # Combined covariance matrix

    # Compute the Z-score and p-value
    l = np.array([[1, -1]])  # Contrast vector for comparing two models
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, delongcov), l.T)).flatten()  # Z-score
    p_value = scipy.stats.norm.sf(abs(z)) * 2  # Two-tailed p-value

    # Return the Z-score and p-value
    z_score = -z[0].item()  # Extract Z-score as a float
    p_value = p_value[0].item()  # Extract p-value as a float

    return z_score, p_value

def bootstrap_auc(y_true, y_prob, n_bootstraps=1000, random_state=42):
    """
    Compute the AUC 95% confidence interval using bootstrapping.
    """
    y_true = np.asarray(y_true)  # Convert to numpy array
    y_prob = np.asarray(y_prob)
    rng = np.random.RandomState(random_state)
    auc_scores = []

    for _ in range(n_bootstraps):
        indices = rng.randint(0, len(y_true), len(y_true))
        y_true_boot = y_true[indices]
        y_prob_boot = y_prob[indices]
        auc_scores.append(roc_auc_score(y_true_boot, y_prob_boot))

    ci_lower = np.percentile(auc_scores, 2.5)
    ci_upper = np.percentile(auc_scores, 97.5)
    return ci_lower, ci_upper


In [2]:
df1 = load_data('~/data/BAH_PRS/version9/conpass.csv')
X = df1.drop(columns=['IHA'])
y = df1['IHA']

smotenc = SMOTENC(categorical_features=[6], random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_resampled, y_train_resampled = smotenc.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_train_scaled = X_train_resampled.copy()
X_test_scaled = X_test.copy()
X_train_scaled.iloc[:, 0:6] = scaler.fit_transform(X_train_scaled.iloc[:, 0:6])
X_test_scaled.iloc[:, 0:6] = scaler.transform(X_test_scaled.iloc[:, 0:6])

tabpfn_model = TabPFNClassifier(n_estimators=32, 
                                device='cpu', 
                                softmax_temperature=0.6, 
                                balance_probabilities=True,
                                random_state=42)
tabpfn_model.fit(X_train_scaled, y_train_resampled)
y_prob1 = tabpfn_model.predict_proba(X_test_scaled)[:, 1]
auc1 = roc_auc_score(y_test, y_prob1)

/tmp/ipykernel_552291/1194473930.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.91927325  1.23730471  1.15205864 ...  1.23730471  0.12910583
 -1.23483125]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_train_scaled.iloc[:, 0:6] = scaler.fit_transform(X_train_scaled.iloc[:, 0:6])
/tmp/ipykernel_552291/1194473930.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-1.14958518  0.4700901  -0.12663237  0.7258283  -1.57581552  1.91927325
 -0.0413863   0.55533617  1.66353505  0.04385976 -0.46761664 -0.12663237
  0.38484403 -0.21187844 -0.89384698 -2.9397526   1.23730471 -1.49056945
  1.15205864  1.06681257  1.32255078  0.7258283   0.12910583 -0.63810878
 -1.40532338 -0.55286271  1.23730471 -0.55286271  0.98156651 -0.29712451
  0.38484403 -1.57581552  0.38484403 -1.49056945 -2.34303013 -1.490569

TabPFNClassifier(balance_probabilities=True, device='cpu', n_estimators=32,
                 random_state=42, softmax_temperature=0.6)

In [3]:
import scipy
logistic_aucs = {}
logistic_models = {}
results = []

for feature in ['Age', 'BMI', 'SBP', 'DBP', 'PAC', 'Renin', 'Sex']:
    X_train_feature = X_train_scaled[[feature]]
    X_test_feature = X_test_scaled[[feature]]
    
    logistic_model = LogisticRegression(random_state=42, solver='liblinear')
    logistic_model.fit(X_train_feature, y_train_resampled)
    
    y_prob_logistic = logistic_model.predict_proba(X_test_feature)[:, 1]
    auc_logistic = roc_auc_score(y_test, y_prob_logistic)
    logistic_aucs[feature] = auc_logistic
    logistic_models[feature] = logistic_model
    
    # Perform DeLong test
    z_score, p_value = delong_test(y_test, y_prob1, y_prob_logistic)
    
    # Compute AUC 95% CI using bootstrapping
    ci_lower, ci_upper = bootstrap_auc(y_test, y_prob_logistic)
    
    # Store results
    results.append({
        'Variable': feature,
        "AUC(95%CI)": f"{auc_logistic:.2f} ({ci_lower:.2f}, {ci_upper:.2f})",
        'Z_value': f"{z_score:.2f}",
        'p_for_delong_test': p_value
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv("/home/luo_wenjin/data/BAH_PRS/version10/ml_dat/delong_test.csv",index = False)
print(results_df)

LogisticRegression(random_state=42, solver='liblinear')

LogisticRegression(random_state=42, solver='liblinear')

LogisticRegression(random_state=42, solver='liblinear')

LogisticRegression(random_state=42, solver='liblinear')

LogisticRegression(random_state=42, solver='liblinear')

LogisticRegression(random_state=42, solver='liblinear')

LogisticRegression(random_state=42, solver='liblinear')

  Variable         AUC(95%CI) Z_value  p_for_delong_test
0      Age  0.57 (0.52, 0.63)  -10.22       1.687827e-24
1      BMI  0.52 (0.45, 0.59)  -10.77       4.555396e-27
2      SBP  0.56 (0.49, 0.63)   -9.36       8.267577e-21
3      DBP  0.58 (0.52, 0.64)   -9.36       7.822438e-21
4      PAC  0.79 (0.73, 0.83)   -6.92       4.501020e-12
5    Renin  0.85 (0.81, 0.88)   -3.90       9.651718e-05
6      Sex  0.64 (0.58, 0.69)   -8.97       3.027039e-19
